In [60]:
#!pip install pandas numpy scikit-learn matplotlib keras tensorflow pyswarms

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from pyswarms.single.global_best import GlobalBestPSO
import pyswarms as ps

# Load and preprocess dataset

In [61]:
# Load the dataset
def load_data():
    # Assuming the dataset is in the same directory and named 'heart_disease_data.csv'
    data = pd.read_csv('heart_statlog_cleveland_hungary_final.csv')
    X = data.drop('target', axis=1)
    y = data['target']
    return X, y

# Preprocess the data
def preprocess(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled

# Implement neural network model

In [66]:
# Define the neural network structure
class NeuralNetwork:
    def __init__(self, n_input, n_hidden, n_output):
        self.w1 = np.random.rand(n_input, n_hidden)
        self.w2 = np.random.rand(n_hidden, n_output)

    def forward(self, X):
        z1 = np.dot(X, self.w1)
        a1 = np.tanh(z1)  # Activation function for hidden layer
        z2 = np.dot(a1, self.w2)
        a2 = np.exp(z2) / np.sum(np.exp(z2), axis=1, keepdims=True)  # Softmax for output
        return a2

    def set_weights(self, weights):
        split = self.w1.size
        self.w1 = weights[:split].reshape(self.w1.shape)
        self.w2 = weights[split:].reshape(self.w2.shape)
    
    def predict(self, X):
        # Make predictions
        proba = self.forward(X)
        return np.argmax(proba, axis=1)

# PSO Setup and Training

In [67]:
# Objective function to minimize
def f_per_particle(weights, nn):
    nn.set_weights(weights)
    y_pred = nn.forward(X_train)
    loss = np.mean((y_train_encoded - y_pred) ** 2)  # Mean squared error
    return loss

# Constraint for PSO
def f(x, nn):
    n_particles = x.shape[0]
    return np.array([f_per_particle(x[i], nn) for i in range(n_particles)])

# Main execution function
def train_network(X, y):
    global X_train, y_train_encoded  # Make these available globally for PSO evaluation
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_train_encoded = np.eye(np.max(y_train) + 1)[y_train]  # One-hot encoding of labels
    
    nn = NeuralNetwork(X_train.shape[1], 10, 2)  # 10 hidden nodes, 2 output nodes
    optimizer = GlobalBestPSO(n_particles=30, dimensions=(nn.w1.size + nn.w2.size), options={'c1': 0.5, 'c2': 0.3, 'w': 0.9})
    best_cost, best_pos = optimizer.optimize(lambda x: f(x, nn=nn), iters=1000, verbose=3)
    
    nn.set_weights(best_pos)
    y_pred = nn.predict(X_test)
    print('Accuracy:', accuracy_score(y_test, y_pred))

# Load data, preprocess and train the network (assuming load_data and preprocess functions are already defined)
X, y = load_data()
X = preprocess(X)
train_network(X, y)

2024-05-19 20:01:48,833 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████████████████████████████████████|1000/1000, best_cost=0.103
2024-05-19 20:03:18,361 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1028248315625541, best pos: [ 1.18949260e+00  2.50310657e-01  2.82950595e-02 -5.99632190e-02
  7.67239183e-01  9.99291968e-02  1.60131139e+00  2.28177834e-01
  2.47571894e+00 -2.62768397e-01  3.47619878e-01  3.67176811e-02
  5.65469143e-01  2.91216287e-01  1.45274234e+00  2.54903502e-01
  5.12465682e-01  1.00783112e+00  1.33062263e-01  8.16110131e-02
  1.07371125e+00 -4.39037425e-01  1.43521289e+00 -2.58760101e-01
 -4.80316673e-01  4.64958744e-01  1.68355826e+00  9.89789730e-01
  6.49832090e-01  2.14955604e-01  9.21869720e-01  5.70447556e-01
 -5.78941965e-01  2.65988522e-01  8.13091033e-01  9.65753345e-01
  5.16

Accuracy: 0.8529411764705882


## Summary

The implementation of various optimization techniques on the heart disease dataset yielded significant results. Among the techniques tested, Particle Swarm Optimization (PSO) achieved the highest accuracy of 85.29%, demonstrating its effectiveness in optimizing neural network weights. This was followed by other methods such as Genetic Algorithm, Simulated Annealing, and Randomized Hill Climbing, each contributing valuable insights into the optimization process. The comparative analysis highlights the strengths and potential of using advanced optimization techniques for improving the performance of neural network models in medical data classification.